# Liver Model Construction: Notebook

In [1]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv


MASSpy version: 0.1.1


In [2]:
maps_dir = os.path.abspath("../massters_model/maps")
data_dir = os.path.abspath("../massters_model/data")
extra_dir=os.path.abspath("../massters_model")

## Add the pathway metabolites and their information to a new COBRA model

In [3]:
# Initiate empty model
trial= cobra.Model()


In [4]:
#Function to load data from the excel sheet
def load_data(filepath, sheet_name):
    """Load Liver data from an excel sheet"""
    df = pd.read_excel(engine='openpyxl',
        io=filepath,
        sheet_name=sheet_name,
        index_col=0)
    return df

In [5]:
# Loading reaction and metabolite data from Excel file, converting them to dictionaries
# data_sheet_met=os.path.join(data_dir,"metabolite_info.xlsx")

# met_info = load_data(
#     filepath=data_sheet_met,
#     sheet_name="Sheet1")
# # .drop("notes", axis=1).drop("annotation", axis=1)
# metabolite_info=met_info.to_dict(orient='index')



# data_sheet_rxn=os.path.join(data_dir,"reactions_info.xlsx")
# rxn_info = load_data(
#     filepath=data_sheet_rxn,
#     sheet_name="Sheet1")
# reactions_info=rxn_info.to_dict(orient='index')

In [6]:
# making dataframe 
csv_rxn = os.path.join(data_dir,"rxn_df")
csv_met = os.path.join(data_dir,"met_df")
rxn_csv_df = pd.read_csv(csv_rxn) 
met_csv_df = pd.read_csv(csv_met,index_col="id")
met_csv_df=met_csv_df.drop("Unnamed: 0",axis=1)
metabolite_info=met_csv_df.to_dict(orient='index')
met_csv_df

,name,charge,formula,compartment
id,,,,
adp_c,ADP C10H12N5O10P2,-3,C10H12N5O10P2,c
atp_c,ATP C10H12N5O13P3,-4,C10H12N5O13P3,c
g6p_c,D-Glucose 6-phosphate,-2,C6H11O9P,c
glc__D_c,D-Glucose,0,C6H12O6,c
h_c,H+,0,H,c
f6p_c,D-Fructose 6-phosphate,-2,C6H11O9P,c
fdp_c,"D-Fructose 1,6-bisphosphate",-4,C6H10O12P2,c
h2o_c,H2O H2O,0,H2O,c
pi_c,Phosphate,0,HO4P,c


In [7]:
# Loading reaction data as JSON file to maintain data types as dictionaries 
with open(r'/home/mass_user/massters_model/data/rxn') as json_file:
    rxn_data = json.load(json_file)

In [8]:
for key, value in metabolite_info.items():
    print(key, value)

adp_c {'name': 'ADP C10H12N5O10P2', 'charge': -3, 'formula': 'C10H12N5O10P2', 'compartment': 'c'}
atp_c {'name': 'ATP C10H12N5O13P3', 'charge': -4, 'formula': 'C10H12N5O13P3', 'compartment': 'c'}
g6p_c {'name': 'D-Glucose 6-phosphate', 'charge': -2, 'formula': 'C6H11O9P', 'compartment': 'c'}
glc__D_c {'name': 'D-Glucose', 'charge': 0, 'formula': 'C6H12O6', 'compartment': 'c'}
h_c {'name': 'H+', 'charge': 0, 'formula': 'H', 'compartment': 'c'}
f6p_c {'name': 'D-Fructose 6-phosphate', 'charge': -2, 'formula': 'C6H11O9P', 'compartment': 'c'}
fdp_c {'name': 'D-Fructose 1,6-bisphosphate', 'charge': -4, 'formula': 'C6H10O12P2', 'compartment': 'c'}
h2o_c {'name': 'H2O H2O', 'charge': 0, 'formula': 'H2O', 'compartment': 'c'}
pi_c {'name': 'Phosphate', 'charge': 0, 'formula': 'HO4P', 'compartment': 'c'}
dhap_c {'name': 'Dihydroxyacetone phosphate', 'charge': -2, 'formula': 'C3H5O6P', 'compartment': 'c'}
g3p_c {'name': 'Glyceraldehyde 3-phosphate', 'charge': -2, 'formula': 'C3H5O6P', 'compartmen

In [9]:
for key, value in rxn_data.items():
    print(key, value)

HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c {'id': 'HEX1', 'name': 'Hexokinase (D-glucose:ATP)', 'metabolites': {'adp_c': 1.0, 'atp_c': -1.0, 'g6p_c': 1.0, 'glc__D_c': -1.0, 'h_c': 1.0}, 'lower_bound': 0.0, 'upper_bound': 1000.0, 'gene_reaction_rule': '3098_AT3 or 80201_AT1 or 2645_AT3 or 3098_AT4 or 3099_AT1 or 2645_AT2 or 3098_AT5 or 2645_AT1 or 3098_AT1 or 3101_AT1 or 3098_AT2'}
PGI: g6p_c <=> f6p_c {'id': 'PGI', 'name': 'Glucose-6-phosphate isomerase', 'metabolites': {'f6p_c': 1.0, 'g6p_c': -1.0}, 'lower_bound': -1000.0, 'upper_bound': 1000.0, 'gene_reaction_rule': '2821_AT1'}
PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c {'id': 'PFK', 'name': 'Phosphofructokinase', 'metabolites': {'fdp_c': 1.0, 'adp_c': 1.0, 'atp_c': -1.0, 'f6p_c': -1.0, 'h_c': 1.0}, 'lower_bound': 0.0, 'upper_bound': 1000.0, 'gene_reaction_rule': '5214_AT1 or 5211_AT2 or (5214_AT1 and 5211_AT1) or (5214_AT1 and 5213_AT1) or 5213_AT1 or (5214_AT1 and 5211_AT2) or 5211_AT1 or (5211_AT1 and 5213_AT1) or (5213_AT1 an

### making cobra model

In [10]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in metabolite_info.items():
    met = cobra.Metabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    trial.add_metabolites(met)

In [11]:
for reaction, info in rxn_data.items():
    print(reaction)
    
    reaction_obj = cobra.Reaction(reaction)
    reaction_obj.id=info['id']
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    trial.add_reaction(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)


HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c
PGI: g6p_c <=> f6p_c
PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
FBP: fdp_c + h2o_c --> f6p_c + pi_c
FBA: fdp_c <=> dhap_c + g3p_c
TPI: dhap_c <=> g3p_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM: 2pg_c <=> 3pg_c
ENO: 2pg_c <=> h2o_c + pep_c
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
DM_nadh: nadh_c --> h_c + nad_c


In [12]:
#View all the reactions that we need build
escher_builder = escher.Builder(
    model=trial,
    map_json=os.path.join(
        maps_dir, ".".join((
            "Glycolysis", "json"))
    ),
    highlight_missing=True)

escher_builder


Builder(highlight_missing=True)

In [13]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {'charge': -1.0}
PGI {}
PFK {'charge': -1.0}
FBP {'charge': 2.0}
FBA {}
TPI {}
GAPD {'charge': -3.0}
PGK {}
PGM {}
ENO {}
PYK {'charge': 1.0}
ATPM {'charge': 1}
DM_nadh {'charge': 1}


In [14]:
trial.metabolites.h_c.charge= 1
# trial.metabolites.h_e.charge= 1
trial.metabolites.pi_c.charge= -2
# trial.metabolites.pi_e.charge= -2
# trial.metabolites.pi_m.charge= -2
# trial.metabolites.h_m.charge= 1
# trial.metabolites.pi_r.charge= -2

In [15]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {}
PGI {}
PFK {}
FBP {}
FBA {}
TPI {}
GAPD {}
PGK {}
PGM {}
ENO {}
PYK {}
ATPM {}
DM_nadh {'charge': 2}


### Add boundary reactions to act as source and sink for glucose and pyruvate

In [16]:
# for met in ['pyr_e', 'h2o_e', 'h_e', 'pi_e','lac__L_e']:
#     rxn = cobra.Reaction('EX_%s' % met)
#     trial.add_reaction(rxn)
#     rxn.add_metabolites({met: -1})
#     if met == 'glc__D_c':
#         rxn.lower_bound = -1 # by convention negative exchange flux = uptake
#     elif met == 'pyr_c':
#         rxn.lower_bound = 0
#     else:
#         rxn.lower_bound = -1000

## Making MASS model of liver

In [17]:
liver= MassModel("Core Model", array_type='DataFrame', dtype=np.int64)
    

In [18]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in metabolite_info.items():
    met = MassMetabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    liver.add_metabolites(met)

In [19]:
cc = ComponentContribution()

In [20]:
for reaction, info in rxn_data.items():
    print(reaction)
    reaction_obj = MassReaction(reaction)
    reaction_obj.id=info['id']
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    liver.add_reactions(reaction_obj)
    temp=info['metabolites']
    
    reaction_obj.add_metabolites(temp)
    
#     cc.search_reaction(reaction)

HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c
PGI: g6p_c <=> f6p_c
PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
FBP: fdp_c + h2o_c --> f6p_c + pi_c
FBA: fdp_c <=> dhap_c + g3p_c
TPI: dhap_c <=> g3p_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM: 2pg_c <=> 3pg_c
ENO: 2pg_c <=> h2o_c + pep_c
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
DM_nadh: nadh_c --> h_c + nad_c


In [71]:
liver.reactions.PGM.reverse_stoichiometry(inplace=True)
liver.reactions.PGM


Reaction identifier,PGM
Name,Phosphoglycerate mutase
Memory address,0x07f0b6e1916d0
Subsystem,
Kinetic Reversibility,True
Stoichiometry,3pg_c <=> 2pg_c 3-Phospho-D-glycerate <=> D-Glycerate 2-phosphate
GPR,
Bounds,"(-1000.0, 1000.0)"


In [73]:
liver.reactions.PGK.reverse_stoichiometry(inplace=True)
liver.reactions.PGK

Reaction identifier,PGK
Name,Phosphoglycerate kinase
Memory address,0x07f0b6e1917d0
Subsystem,
Kinetic Reversibility,True
Stoichiometry,13dpg_c + adp_c <=> 3pg_c + atp_c 3-Phospho-D-glyceroyl phosphate + ADP C10H12N5O10P2 <=> 3-Phospho-D-glycerate + ATP C10H12N5O13P3
GPR,
Bounds,"(-1000.0, 1000.0)"


In [22]:
def _replace_compartment_id(self, sep="_", new_cid=None):
        """Remove or replace the compartment from the ID str of the metabolite."""
        # Return the identifier as is if no compartment is defined.
        if not self.compartment:
            return self.id

        new_id = self.id.rpartition(sep + str(self.compartment))[0]
        if new_cid is not None:
            new_id = sep.join((new_id, new_cid))
        
        return new_id

In [74]:
## Creating reactions for gluconeogenesis

# G6P = MassReaction(
#     "G6P",
#     name="glucose 6-phosphatase",
#     reversible=False)
# G6P.add_metabolites({
    
#     liver.metabolites.get_by_id('g6p_c'): -1,
#     liver.metabolites.get_by_id('h2o_c'): -1,
#     liver.metabolites.get_by_id('pi_c') : 1,
#     liver.metabolites.get_by_id('glc__D_c'): 1})



# liver.add_reactions([
#    G6P ])


## Lehningers Table 14-3 
# the reaction is to be counted twice, 
# because two three-carbon precursors are required to make a molecule of glucose
# liver.reactions.PCm + liver.reactions.PCm 

# liver.reactions.PEPCKm + liver.reactions.PEPCKm


# atpase_reaction = cc.parse_reaction_formula(
#     "bigg.metabolite:atp + bigg.metabolite:h2o = "
#     "bigg.metabolite:adp + bigg.metabolite:pi"
# )


In [87]:


liver.reactions.PYK.reversible=False
liver.reactions.HEX1.reversible=False
reaction_str= []

cc = ComponentContribution()

# optional: changing the aqueous environment parameters
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3.0)
cc.ionic_strength = Q_("0.25M")
cc.temperature = Q_("298.15K")




for reaction in liver.reactions:

    if reaction.id == 'PYK' or reaction.id == 'HEX1':
        rxn_name = reaction.id
        
        reaction_str = reaction.reaction.split(" --> ")
        reactants = reaction_str[0]
        reactants = reactants.split(" + ")
#         print(reactants)
        for i in range(len(reactants)):
            string = reactants[i]
            l = len(string)
            reactants[i] = string[:l-2]
            reactants[i]= str("bigg.metabolite:" + reactants[i])
        reactants_string= " + ".join(reactants)
        reactants_string = str(reactants_string + " = ")
#         print(reactants_string)
        
        
        products = reaction_str[1]
        products = products.split(" + ")
#         print(products)
        for i in range(len(products)):
            string = products[i]
            l = len(string)
            products[i] = string[:l-2]            
            products[i]= "bigg.metabolite:" + products[i]
        products_string= ' + '.join(products) 
#         print(products_string)
        reaction_to_parse = reactants_string + products_string
#         print(str(reaction_to_parse))     
        rxn_parsed = cc.parse_reaction_formula(reaction_to_parse)
        dG0_prime = cc.standard_dg_prime(rxn_parsed)
        print(rxn_name)
        print(f"ΔG'° = {dG0_prime}")

        dGm_prime = cc.physiological_dg_prime(rxn_parsed)
        print(f"ΔG'm = {dGm_prime}")
        
        
    
    else:
        rxn_name = reaction.id        
        reaction_str = reaction.reaction.split(" <=> ")
        reactants = reaction_str[0]
        reactants = reactants.split(" + ")
#         print(reactants)        
        for i in range(len(reactants)):
            string = reactants[i]
            l = len(string)
            reactants[i] = string[:l-2]
            reactants[i]= "bigg.metabolite:" + reactants[i]
        reactants_string= " + ".join(reactants)
        reactants_string = str(reactants_string + " = ")  
#         print(reactants_string)    
        
        
        
        products = reaction_str[1]
        products = products.split(" + ")
#         print(products)
        for i in range(len(products)):
            string = products[i]
            l = len(string)
            products[i] = string[:l-2]            
            products[i]= "bigg.metabolite:" + products[i]
        products_string= ' + '.join(products) 
#         print(products_string)
        reaction_to_parse = reactants_string + products_string
#         print(str(reaction_to_parse))             
        rxn_parsed = cc.parse_reaction_formula(reaction_to_parse)
        print(rxn_name)
        dG0_prime = cc.standard_dg_prime(rxn_parsed)
        print(f"ΔG'° = {dG0_prime}")

        dGm_prime = cc.physiological_dg_prime(rxn_parsed)
        print(f"ΔG'm = {dGm_prime}")        


    

#     cc = ComponentContribution()
#     cc.parse_reaction_formula((reaction_str)

HEX1
ΔG'° = (-19.9 +/- 0.4) kilojoule / mole
ΔG'm = (-19.9 +/- 0.4) kilojoule / mole
PGI
ΔG'° = (2.6 +/- 0.4) kilojoule / mole
ΔG'm = (2.6 +/- 0.4) kilojoule / mole
PFK
ΔG'° = (-17.1 +/- 0.7) kilojoule / mole
ΔG'm = (-17.1 +/- 0.7) kilojoule / mole
FBP
ΔG'° = (-12.0 +/- 0.7) kilojoule / mole
ΔG'm = (-29.2 +/- 0.7) kilojoule / mole
FBA
ΔG'° = (23.0 +/- 0.5) kilojoule / mole
ΔG'm = (5.9 +/- 0.5) kilojoule / mole
TPI
ΔG'° = (5.6 +/- 0.5) kilojoule / mole
ΔG'm = (5.6 +/- 0.5) kilojoule / mole
GAPD
ΔG'° = (1.8 +/- 0.4) kilojoule / mole
ΔG'm = (18.9 +/- 0.4) kilojoule / mole
PGK
ΔG'° = (-19.5 +/- 0.4) kilojoule / mole
ΔG'm = (-19.5 +/- 0.4) kilojoule / mole
PGM
ΔG'° = (4.5 +/- 0.4) kilojoule / mole
ΔG'm = (4.5 +/- 0.4) kilojoule / mole
ENO
ΔG'° = (-3.81 +/- 0.30) kilojoule / mole
ΔG'm = (-3.81 +/- 0.30) kilojoule / mole
PYK
ΔG'° = (-25.6 +/- 0.4) kilojoule / mole
ΔG'm = (-25.6 +/- 0.4) kilojoule / mole
ATPM
ΔG'° = (-29.14 +/- 0.30) kilojoule / mole
ΔG'm = (-46.26 +/- 0.30) kilojoule / mole
D

In [76]:
for r in liver.reactions:
    print(r.reaction)

atp_c + glc__D_c --> adp_c + g6p_c + h_c
g6p_c <=> f6p_c
atp_c + f6p_c <=> adp_c + fdp_c + h_c
fdp_c + h2o_c <=> f6p_c + pi_c
fdp_c <=> dhap_c + g3p_c
dhap_c <=> g3p_c
g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
13dpg_c + adp_c <=> 3pg_c + atp_c
3pg_c <=> 2pg_c
2pg_c <=> h2o_c + pep_c
adp_c + h_c + pep_c --> atp_c + pyr_c
atp_c + h2o_c <=> adp_c + h_c + pi_c
nadh_c <=> h_c + nad_c


### Adding exchange reactions

In [25]:
# for met in [
# #     'co2_m', 'gdp_m', 'gtp_m', 'pi_m', 'hco3_m','h_m','atp_c','atp_m'
#     'glc__D_c','h_c','h2o_c','pi_c','pyr_c'
#            ]:
#     rxn = MassReaction('EX_%s' % met)
#     liver.add_reaction(rxn)
#     rxn.add_metabolites({met: -1})
#     if met == 'glc__D_c':
#         rxn.lower_bound = -1 # by convention negative exchange flux = uptake
#     elif met == 'pyr_c':
#         rxn.lower_bound = 0
#     else:
#         rxn.lower_bound = -1000
        
# liver.reactions.L_LACt2r.lower_bound=-1
# liver.reactions.PYRt2.lower_bound=-1
# # liver.reactions.GLCt1.lower_bound=-1
# liver.reactions.Ht.lower_bound=-1000
# liver.reactions.H2Ot.lower_bound=-1000
# liver.reactions.PIt.lower_bound=-1000



In [26]:
liver.metabolites.h_c.charge= 1
# liver.metabolites.h_e.charge= 1
liver.metabolites.pi_c.charge= -2
# liver.metabolites.pi_e.charge= -2
# trial.metabolites.pi_m.charge= -2
# trial.metabolites.h_m.charge= 1

In [27]:
# liver.reactions.GLCt1

In [28]:
for r in liver.reactions:
    print(r.id,  r.check_mass_balance())
print(liver.reactions.FBA.check_mass_balance())

HEX1 {}
PGI {}
PFK {}
FBP {}
FBA {}
TPI {}
GAPD {}
PGK {}
PGM {}
ENO {}
PYK {}
ATPM {}
DM_nadh {'charge': 2}
{}


In [29]:
liver.stoichiometric_matrix

,HEX1,PGI,PFK,FBP,FBA,TPI,GAPD,PGK,PGM,ENO,PYK,ATPM,DM_nadh
adp_c,1,0,1,0,0,0,0,1,0,0,-1,1,0
atp_c,-1,0,-1,0,0,0,0,-1,0,0,1,-1,0
g6p_c,1,-1,0,0,0,0,0,0,0,0,0,0,0
glc__D_c,-1,0,0,0,0,0,0,0,0,0,0,0,0
h_c,1,0,1,0,0,0,1,0,0,0,-1,1,1
f6p_c,0,1,-1,1,0,0,0,0,0,0,0,0,0
fdp_c,0,0,1,-1,-1,0,0,0,0,0,0,0,0
h2o_c,0,0,0,-1,0,0,0,0,0,1,0,-1,0
pi_c,0,0,0,1,0,0,-1,0,0,0,0,1,0
dhap_c,0,0,0,0,1,-1,0,0,0,0,0,0,0


In [30]:
liver.objective = 'ATPM'
flux_solution = liver.optimize()
flux_solution.fluxes


HEX1       0.0
PGI        0.0
PFK        0.0
FBP        0.0
FBA        0.0
TPI        0.0
GAPD       0.0
PGK        0.0
PGM        0.0
ENO        0.0
PYK        0.0
ATPM       0.0
DM_nadh    0.0
Name: fluxes, dtype: float64

## Escher FBA

In [31]:
#Escher FBA: shows the solutions on the map
flux_data = {
    reaction: flux
    for reaction, flux in flux_solution.fluxes.items()}

#view all the reactions that we need build
escher_builder = escher.Builder(
    model=liver,
    map_json=os.path.join(
        maps_dir, ".".join((
            "Glycolysis", "json"))
    ), reaction_data=flux_data)

# Display map in notebook
escher_builder

Builder(reaction_data={'HEX1': 0.0, 'PGI': 0.0, 'PFK': 0.0, 'FBP': 0.0, 'FBA': 0.0, 'TPI': 0.0, 'GAPD': 0.0, '…

### Set fluxes

In [32]:
reaction_list = liver.reactions
for reaction, flux in flux_solution.fluxes.items():
    reaction = liver.reactions.get_by_id(reaction)
    reaction.steady_state_flux = flux 
# have to check conversions    
# * 1278.605 / 1000 # mmol/gdW*hr --> mmol/L * hr ---> mol/L*hr
    print("{0}: {1}".format(reaction.flux_symbol_str,
                            reaction.steady_state_flux))

v_HEX1: 0.0
v_PGI: 0.0
v_PFK: 0.0
v_FBP: 0.0
v_FBA: 0.0
v_TPI: 0.0
v_GAPD: 0.0
v_PGK: 0.0
v_PGM: 0.0
v_ENO: 0.0
v_PYK: 0.0
v_ATPM: 0.0
v_DM_nadh: 0.0


### Set Keqs

In [33]:
# pip install equilibrator-api

In [34]:
# cc = ComponentContribution()

In [35]:
# pip install altair vega_datasets

## Getting Gibbs free energies of reactions

In [36]:
cc = ComponentContribution()

# # optional: changing the aqueous environment parameters
# cc.p_h = Q_(7.4)
# cc.p_mg = Q_(3.0)
# cc.ionic_strength = Q_("0.25M")
# cc.temperature = Q_("298.15K")
# cc.search_reaction


In [37]:
## Reading Gibbs Energies from Lehningers
ge = os.path.join(data_dir,"Gibbs_energies.csv")
ge_df = pd.read_csv(ge) 
# met_csv_df = pd.read_csv(csv_met,index_col="id")
ge_df = ge_df.drop("string", axis=1)
# metabolite_info=met_csv_df.to_dict(orient='index')
# met_csv_df
ge_df

,reaction,Gibbs Energies (Lehningers Table 14-2)( kJ/mol) min,Max
0,HEX1,-33.4,-33.4
1,PGI,0.0,25.0
2,PFK,-22.2,-22.2
3,FBP,NaN,NaN
4,FBA,-6.0,0.0
5,TPI,0.0,4.0
6,GAPD,-2.0,2.0
7,PGK,0.0,2.0
8,PGM,0.0,0.8
9,ENO,0.0,3.3


In [38]:
liver.metabolites.atp_c.summary()

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


In [39]:
from cobra.flux_analysis import flux_variability_analysis

In [40]:
flux_variability_analysis(liver, liver.reactions[:])

,minimum,maximum
HEX1,0.0,0.0
PGI,0.0,0.0
PFK,0.0,0.0
FBP,0.0,0.0
FBA,0.0,0.0
TPI,0.0,0.0
GAPD,0.0,0.0
PGK,0.0,0.0
PGM,0.0,0.0
ENO,0.0,0.0


In [41]:
liver.optimize()
liver.summary(fva=0.95)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
Metabolite,Reaction,Flux,Range,C-Number,C-Flux


In [42]:
cobra.flux_analysis.flux_variability_analysis(
    liver, liver.reactions[:], fraction_of_optimum=0.9)

,minimum,maximum
HEX1,0.0,0.0
PGI,0.0,0.0
PFK,0.0,0.0
FBP,0.0,0.0
FBA,0.0,0.0
TPI,0.0,0.0
GAPD,0.0,0.0
PGK,0.0,0.0
PGM,0.0,0.0
ENO,0.0,0.0


### Export

In [43]:
save_json_mass_model(
    mass_model=liver,
    filename=os.path.join(extra_dir, liver.id + ".json"))